<a href="https://colab.research.google.com/github/Soha85/MyKaggle/blob/main/Simple_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q torch transformers transformers langchain sentence-transformers tqdm openpyxl openai pandas datasets langchain-community ragatouille bitsandbytes bert_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 483.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 27.8 MB/s eta 0:00:00
   ━━

In [2]:
from google.colab import drive
import pandas as pd
drive.mount('/content/gdrive')
data = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/train-squad.csv')

Mounted at /content/gdrive


In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Initialize the text splitter
# You can adjust chunk_size and chunk_overlap as needed
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,      # Max size of each chunk
    chunk_overlap=50,    # Overlap between chunks to maintain context
    add_start_index=True, # This helps track where each chunk starts in the original text
    separators=["\n\n", "\n", ".", " ", ""]
)

# Create a list to store the processed chunks of documents
docs_processed = []

# Loop through each question and text in the DataFrame
for index, row in data[['question', 'text']].head(1000).iterrows():
    question = row['question']
    text = row['text']

    # Split the text into chunks
    chunks = text_splitter.split_text(text)

    # Each chunk can be associated with the corresponding question
    for chunk in chunks:
        docs_processed.append({
            'question': question,
            'text_chunk': chunk,
            'chunk_start': text.find(chunk)
        })

# Check the first few processed chunks
print(docs_processed[:5])


[{'question': 'When did Beyonce start becoming popular?', 'text_chunk': 'in the late 1990s', 'chunk_start': 0}, {'question': 'What areas did Beyonce compete in when she was growing up?', 'text_chunk': 'singing and dancing', 'chunk_start': 0}, {'question': "When did Beyonce leave Destiny's Child and become a solo singer?", 'text_chunk': '2003', 'chunk_start': 0}, {'question': 'In what city and state did Beyonce  grow up? ', 'text_chunk': 'Houston, Texas', 'chunk_start': 0}, {'question': 'In which decade did Beyonce become famous?', 'text_chunk': 'late 1990s', 'chunk_start': 0}]


In [4]:
from transformers import BertTokenizer, BertModel
import torch

# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Function to embed text chunks
def embed_text(text_chunk):
    inputs = tokenizer(text_chunk, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():  # Disable gradient calculation for faster processing
        outputs = model(**inputs)
    # Get the embeddings of the [CLS] token (first token)
    embeddings = outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()
    return embeddings

# Embed all the text chunks
for doc in docs_processed:
    text_chunk = doc['text_chunk']
    doc['embedding'] = embed_text(text_chunk)  # Add the embedding to each document


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [5]:
import faiss
import numpy as np

# Initialize a FAISS index (index for dense vectors, L2 distance)
embedding_dim = docs_processed[0]['embedding'].shape[0]  # Get the embedding dimension
index = faiss.IndexFlatL2(embedding_dim)  # L2 distance (Euclidean distance)

# Prepare the embeddings and keep track of the associated metadata
embeddings = []
metadata = []

# Add each embedding to FAISS and store its metadata
for doc in docs_processed:
    embedding = doc['embedding']
    embeddings.append(embedding)
    metadata.append({
        'question': doc['question'],
        'text_chunk': doc['text_chunk'],
        'chunk_start': doc['chunk_start']
    })

# Convert embeddings to a numpy array and add to FAISS index
embeddings = np.array(embeddings).astype('float32')  # FAISS expects float32
index.add(embeddings)

# Check the size of the FAISS index (should match the number of chunks)
print(f"Number of embeddings in the index: {index.ntotal}")


Number of embeddings in the index: 1000


In [6]:
# Function to embed a query (similar to how we embed the text chunks)
def embed_query(query):
    inputs = tokenizer(query, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    query_embedding = outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()
    return query_embedding

# Example query
query,answer= data[['question','text']].sample(n=1).iloc[0]
print(f"Query: {query}")
print(f"Answer: {answer}")
# Embed the query
query_embedding = embed_query(query).astype('float32')  # FAISS expects float32

# Perform similarity search in FAISS (k=5 means retrieving the 5 most relevant chunks)
k = 5
distances, indices = index.search(np.array([query_embedding]), k)

# Retrieve and display the top 5 most relevant chunks
print("Top 5 relevant chunks:")
for i, idx in enumerate(indices[0]):
    result = metadata[idx]
    print(f"\nResult {i+1}:")
    print(f"Question: {result['question']}")
    print(f"Text chunk: {result['text_chunk']}")
    print(f"Chunk start position in text: {result['chunk_start']}")

Query: Before insulin, what was the life expectancy of diabetics?
Answer: only a few months
Top 5 relevant chunks:

Result 1:
Question: What was Destiny's Child's first major song hit?
Text chunk: No, No, No
Chunk start position in text: 0

Result 2:
Question: "Charlie's Angels" featured which single from the band members?
Text chunk: Independent Women Part I
Chunk start position in text: 0

Result 3:
Question: What album did the Spanish songs come from?
Text chunk: B'Day.
Chunk start position in text: 0

Result 4:
Question: Destiny's Child song, Killing Time, was included in which film's soundtrack?
Text chunk: Men in Black.
Chunk start position in text: 0

Result 5:
Question: Beyonce had singers in the background known by the name as?
Text chunk: The Mamas
Chunk start position in text: 0


In [7]:
from transformers import pipeline

# Initialize the text generation pipeline with DistilGPT-2
generation_pipeline = pipeline('text-generation', model='distilgpt2')

# Set a prompt template to pass the retrieved chunks to the generation model
prompt_template = "Context: {context}\nQuestion: {question}\nAnswer:"

# Function to generate answers based on retrieved chunks
def generate_answer(question, retrieved_chunks):
    # Combine the retrieved chunks to form the context
    context = " ".join([chunk['text_chunk'] for chunk in retrieved_chunks])

    # Create the prompt by injecting the context and question
    prompt = prompt_template.format(context=context, question=question)

    # Generate the answer using the text generation model
    answer = generation_pipeline(prompt, max_length=100, num_return_sequences=1)
    return answer[0]['generated_text']

# Example usage:
query,answer= data[['question','text']].sample(n=1).iloc[0]
print(f"Query: {query}")
print(f"Answer: {answer}")

# Step 1: Embed the query
query_embedding = embed_query(query).astype('float32')

# Step 2: Perform similarity search in FAISS
k = 5
distances, indices = index.search(np.array([query_embedding]), k)

# Step 3: Retrieve the top k relevant chunks
retrieved_chunks = [metadata[idx] for idx in indices[0]]

# Step 4: Generate the answer based on the retrieved chunks
generated_answer = generate_answer(query, retrieved_chunks)
print("Generated Answer:\n", generated_answer)


config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Query: What do Jehovah Witnesses remain politically?
Answer: neutral
Generated Answer:
 Context: Independent Women Part I it is categorically untrue Demand A Plan B'Day. Life Is But a Dream
Question: What do Jehovah Witnesses remain politically?
Answer: I'm not trying to make you uncomfortable because I want your life. In my time, I've been a pro-choice advocate for religious freedom and I had a lot of work with me when it came time to decide on who would do so. At the same time, I also consider myself to be a feminist of


In [8]:
import bert_score

# Example BERTScore evaluation
P, R, F1 = bert_score.score([generated_answer.split('Answer:')[1]], [answer], lang="en", model_type="bert-base-uncased")

# Output the BERTScore
print(f"BERTScore - Precision: {P.mean().item()}, Recall: {R.mean().item()}, F1: {F1.mean().item()}")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


BERTScore - Precision: 0.1797114461660385, Recall: 0.6247311234474182, F1: 0.2791282534599304


In [9]:
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from transformers import pipeline
from langchain.docstore.document import Document
from langchain import PromptTemplate

# 1. Create embeddings and store documents
embeddings = SentenceTransformerEmbeddings(model_name="all-mpnet-base-v2")

# Convert docs_processed to a list of Document objects
docs_for_faiss = [
    Document(page_content=doc['text_chunk'], metadata={k: v for k, v in doc.items() if k != 'text_chunk'})
    for doc in docs_processed
]

# Now use docs_for_faiss with FAISS.from_documents
db = FAISS.from_documents(docs_for_faiss, embeddings)

# 2. Load the language model
generation_pipeline = pipeline('text-generation', model='distilgpt2')
# Pass max_length to the HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=generation_pipeline, model_kwargs={"max_length": 200, "max_new_tokens": 150})

# 3. Create the RetrievalQA chain
# Define the prompt template using PromptTemplate
prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="Context: {context}\nQuestion: {question}\nAnswer:"
)

# Remove max_length from chain_type_kwargs
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(),
    chain_type_kwargs={
        "prompt": prompt_template,  # Use the PromptTemplate object
    }
)

# 4. Run the RAG pipeline
# Example usage:
query, answer = data[['question', 'text']].sample(n=1).iloc[0]
print(f"Query: {query}")
print(f"Answer: {answer}")
result = qa.run(query)
print(result)

<ipython-input-9-55dcf61ba129>:10: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name="all-mpnet-base-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<ipython-input-9-55dcf61ba129>:24: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=generation_pipeline, model_kwargs={"max_length": 200, "max_new_tokens": 150})
<ipython-input-9-55dcf61ba129>:48: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa.run(query)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Query: What reasoning would cause a country to be referred to as the "51st state" in a positive way?
Answer: a region or territory is so aligned, supportive, and conducive with the United States, that it is like a U.S. state
Context: America the Beautiful

America the Beautiful

America the Beautiful

God Bless the USA
Question: What reasoning would cause a country to be referred to as the "51st state" in a positive way?
Answer: No
